In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [2]:
# 1. Load Documents
loader = PyPDFLoader("acko-group-health-insurance-policy-policy-wordings_copy.pdf")   # apna text file daloo
documents = loader.load()

In [3]:
# 2. Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [4]:
# 3. Embeddings + VectorStore
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.environ["GOOGLE_API_KEY"])
vectorstore = Chroma.from_documents(docs, embeddings)

In [5]:
# 4. Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
#👉 Ye ek search tool banata hai jo query ke embeddings ke basis pe similar docs nikal ke dega.

In [6]:
# 5. LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])
#👉 Ye tumhara language model (ChatGPT) hai jo final answer generate karega.

In [7]:
# 6. RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)
#👉 Retrieval + LLM ko ek pipeline (chain) me jod diya gaya:

   #User query →

    #Retriever → relevant chunks find karega (PDF ke andar se)

    #LLM (GPT) → retrieved text + query se final answer generate karega

#⚡chain_type="stuff" matlab simple concatenation of retrieved docs.

In [8]:
#Generation
result = qa_chain.invoke("What is the claim process for health insurance?")
print("Answer:", result)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.ge

KeyboardInterrupt: 